In [21]:
from pickle import load
from numpy import argmax

from keras.models import Model
from tensorflow.python.keras.models import load_model
from keras_preprocessing.image import load_img, img_to_array
from keras_preprocessing.sequence import pad_sequences

from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import preprocess_input as preprocess_input_i
from keras.applications.vgg16 import preprocess_input as preprocess_input_v

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

In [23]:
curr_folder = "D:/YandexDisk/datasets/"
path_tokenizer = curr_folder + "ru-12k-tokenizer-train.pkl"
tokenizer = load(open(path_tokenizer, 'rb'))
max_words = 22

vgg16_extractor = VGG16()
vgg16_extractor = Model(inputs=vgg16_extractor.inputs, outputs=vgg16_extractor.layers[-2].output)
vgg16_target_size = 224

inception_extractor = InceptionV3(weights="imagenet")
inception_extractor = Model(inputs=inception_extractor.inputs, outputs=inception_extractor.layers[-2].output)
inception_target_size = 299

v0 = "D:/models/vgg16/model-0.h5"
v1 = "D:/models/vgg16/model-1.h5"
v2 = "D:/models/vgg16/model-2.h5"
v3 = "D:/models/vgg16/model-3.h5"
v4 = "D:/models/vgg16/model-4.h5"
v5 = "D:/models/vgg16/model-5.h5"
v6 = "D:/models/vgg16/model-6.h5"
v7 = "D:/models/vgg16/model-7.h5"
v8 = "D:/models/vgg16/model-8.h5"
v9 = "D:/models/vgg16/model-9.h5"
v10 = "D:/models/vgg16/model-10.h5"
v11 = "D:/models/vgg16/model-11.h5"
v12 = "D:/models/vgg16/model-12.h5"
v13 = "D:/models/vgg16//model-13.h5"
v14 = "D:/models/vgg16/model-14.h5"
v15 = "D:/models/vgg16/model-15.h5"
v16 = "D:/models/vgg16/model-16.h5"
v17 = "D:/models/vgg16/model-17.h5"
v18 = "D:/models/vgg16/model-18.h5"
v19 = "D:/models/vgg16/model-19.h5"

i0 = "D:/models/inception/model-0.h5"
i1 = "D:/models/inception/model-1.h5"
i2 = "D:/models/inception/model-2.h5"
i3 = "D:/models/inception/model-3.h5"
i4 = "D:/models/inception/model-4.h5"
i5 = "D:/models/inception/model-5.h5"
i6 = "D:/models/inception/model-6.h5"
i7 = "D:/models/inception/model-7.h5"
i8 = "D:/models/inception/model-8.h5"
i9 = "D:/models/inception/model-9.h5"
i10 = "D:/models/inception/model-10.h5"
i11 = "D:/models/inception/model-11.h5"
i12 = "D:/models/inception/model-12.h5"
i13 = "D:/models/inception/model-13.h5"
i14 = "D:/models/inception/model-14.h5"
i15 = "D:/models/inception/model-15.h5"
i16 = "D:/models/inception/model-16.h5"
i17 = "D:/models/inception/model-17.h5"
i18 = "D:/models/inception/model-18.h5"
i19 = "D:/models/inception/model-19.h5"

v_models = [v0, v1, v2, v3, v4, v5, v6, v7, v8, v9,
            v10, v11, v12, v13, v14, v15, v16, v17, v18, v19]

i_models = [i0, i1, i2, i3, i4, i5, i6, i7, i8, i9,
            i10, i11, i12, i13, i14, i15, i16, i17, i18, i19]

In [22]:
def map_int_to_word(integer, tokenizer):
    for word, idx in tokenizer.word_index.items():
        if idx == integer:
            return word

    return None

def generate_caption(model, tokenizer, max_words, image):
    in_text = 'startseq'

    for i in range(max_words):
        seq = tokenizer.texts_to_sequences([in_text])[0]
        seq = pad_sequences([seq], maxlen=max_words)

        y_hat = model.predict([image,seq], verbose=0)
        y_hat = argmax(y_hat)

        word = map_int_to_word(y_hat, tokenizer)
        if word is None:
            break

        in_text += ' ' + word

        if word == 'endseq':
            break

    return in_text

def print_image(path_image):
    image = mpimg.imread(path_image)
    plt.imshow(image)
    plt.show()

def extract_features(path_image, extractor, target_size, model_type):
    image = load_img(path_image, target_size=(target_size, target_size))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    if model_type == "vgg16":
        image = preprocess_input_v(image)
    elif model_type == "inception":
        image = preprocess_input_i(image)

    feature = extractor.predict(image, verbose=0)

    return feature

def generate_and_print_captions(path_image, extractor, target_size, models_paths, model_type):
    image_features = extract_features(path_image, extractor, target_size, model_type)

    for i, path in enumerate(models_paths):
        model = load_model(path)
        caption = generate_caption(model, tokenizer, max_words, image_features)
        print(str(i) + ' ' + caption)